#### Imports, configurations, constant/variables

In [1]:
# impòrts

In [2]:
import pandas
from google.colab import files

In [3]:
from pandas.core.indexes.datetimes import date

In [4]:
#from pandas.core.reshape.tile import to_datetime

In [5]:
#from numpy import datetime64

In [6]:
#from datetime import datetime

In [7]:
#import dateutil.parser

In [8]:
from dateutil.relativedelta import relativedelta

In [9]:
pandas.set_option('display.max_rows', None)

#### Upload Data

##### Upload files

In [10]:
uploaded = files.upload()     # To upload .csv files to colab storage

Saving benchmark.csv to benchmark.csv
Saving candidates_df.csv to candidates_df.csv
Saving competitor_dates.csv to competitor_dates.csv


In [11]:
!ls -lh

total 14M
-rw-r--r-- 1 root root 348K Nov 28 21:07 benchmark.csv
-rw-r--r-- 1 root root  13M Nov 28 21:07 candidates_df.csv
-rw-r--r-- 1 root root  91K Nov 28 21:07 competitor_dates.csv
drwxr-xr-x 1 root root 4.0K Nov 22 00:14 sample_data


##### Create candidates dataFrame

In [12]:
# Create dataframe 
candidates_df_data = pandas.read_csv('candidates_df.csv', parse_dates=['date'],
                                     dtype={'cluster_id':str, 'stage_name':str})
to_drop =['country', 'cluster', 'stage_name_lag_1', 'business_unit','ther_area_fact', 'prevalence']
candidates = candidates_df_data.drop(columns=to_drop) #Here are the data with cluster, date, stages and volume

##### Other dataFrames

In [13]:
#benchmark_data = pandas.read_csv('benchmark.csv')

In [14]:
#competitor_dates_data = pandas.read_csv('competitor_dates.csv')

In [15]:
#indication_dates_data = pandas.read_csv('indication_dates.csv')

In [16]:
#launch_dates_data = pandas.read_csv('launch_dates.csv')

In [17]:
#population_data = pandas.read_csv('population.csv')

In [18]:
#regulatory_designations_data = pandas.read_csv('regulatory_designations.csv')

# Functions

##### Normalization Coefficient to adjust slopes of derivative.

Normalization Coefficient to adjust slopes of derivative.
```
For all rows of each cluster_id determine Max and Min
with ValueRef, then return coefficient to apply slope calculation
Example:

cluster_id	date	stage_name	volume
...
ID_65	2015-07-01	Growth	411482324,4606376
ID_65	2015-08-01	Growth	414471300,18311197
ID_65	2015-09-01	Growth	468771025,808063
ID_65	2015-10-01	Growth	424932715,21177226
ID_65	2015-11-01	Growth	439628512,5139379
ID_65	2015-12-01	Growth	407746104,80754465
ID_65	2016-01-01	Growth	427174447,003628
ID_65	2016-02-01	Growth	339995988,43145907
ID_65	2016-03-01	Growth	415965788,04434913
ID_65	2016-04-01	Growth	433650561,06898916
ID_65	2016-05-01	Growth	428170772,24445283
ID_65	2016-06-01	Growth	470514594,9795063
ID_65	2016-07-01	Growth	426925365,69342184
ID_65	2016-08-01	Maturity	589575461,2580684
ID_65	2016-09-01	Maturity	547729801,1434274
ID_65	2016-10-01	Maturity	513854742,9553846
ID_65	2016-11-01	Maturity	535524816,94332385
ID_65	2016-12-01	Maturity	622952356,825699
ID_65	2017-01-01	Maturity	438632187,2731131
ID_65	2017-02-01	Maturity	530294109,42899364
ID_65	2017-03-01	Maturity	631421121,3727096
ID_65	2017-04-01	Maturity	550967858,176108
ID_65	2017-05-01	Maturity	557443972,2414691
ID_65	2017-06-01	Maturity	689207985,3405474
ID_65	2017-07-01	Maturity	567158143,3395108
ID_65	2017-08-01	Maturity	618219811,9317812
ID_65	2017-09-01	Maturity	578864964,9192022
...
```
id0065.svg

In [19]:
# Normalization Coefficient to adjust slopes of derivative.
# For all rows of each cluster_id determine Max and Min
# with ValueRef, then return coefficient to apply slope calculation
# also can provide mean of values.
def normalizeSales(salesArray,valueRef):
  max = salesArray.max()
  min = salesArray.min()
  range = int(max - min)
  realMean = salesArray.mean()
  print('Range for all registers to fit valueref:', range)
  print('real mean calculated', realMean)
  norm_mean = (realMean/range)*valueRef
  salesNormCoefficient = valueRef/range
  return salesNormCoefficient, norm_mean

##### Create product list from candidates

In [20]:
# prodList is a dataFrame of unique cluster_id list
# with transition date, is_transition(YES/NO)
# with date of transition, and a forecast value for this transition date
# pseudoST(Slopes list), pseudoLT(Slopes list),

In [21]:
prodList = pandas.DataFrame(
    { 'cluster_id': candidates['cluster_id'].unique(), # unique cluster_id list
     'transition_date': date, # transition date point forecasted for first submission
     'is_transition': str, # if YES or NO transitioned to mature
     'date': date, # transition date point forecasted for second submission
     'forecast': int, # forecast value for this transition date, second submission
     'idxGroup': list, # list of indexes that are included on this cluster_id
     'idxLtrans': list, # Indexes of rows that transitions due labelling at dataset
     'idxPtrans':list, # Indexes of rows that transitions due parametric determination
     'idxST_l': list, # short term calculation ranges for transitional indexes (labelling)
     'idxLT_l':list, # long term calculation ranges for transitional indexes (parametric)
     'diffST_l':list, # short term calculation pseudoderivative for calculation ranges (labelling)
     'diffLT_l':list # long term calculation pseudoderivative for calculation ranges (labelling)
    })

##### Create a submission1 csv file from prodList

In [22]:
header = ['cluster_id', 'transition_date', 'is_transition']
prodList.to_csv('metric_one.csv', columns = header)

In [23]:
files.download('metric_one.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##### Create a submission2 csv file from prodList

In [24]:
header = ['cluster_id', 'date', 'forecast']
prodList.to_csv('metric_two.csv', columns = header)

In [25]:
files.download('metric_two.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##### Explore dataframe for each cluster_id value
```
# From candidate dataFrame, create prodList unique rows for each prod_id
# Going over prodList to extract for each prod_id:
# |-> idxGroup(list of indexes for this prod_id),
# |-> idxLtrans(list indexes of transition labels for this prod_id)
# |-> idxST_l(lists indexes of short term analyze ranges, one list for each index of transition)
# |-> idxLT_l(lists indexes of long term analyze ranges, one list for each index of transition)
```

In [26]:
def scanProdId(prod_id, test = False):
  global candidates
  global prodlist
  global group
  count = 0
  last_stage = None
  DiffList = []
  idxTransitions = []
  lastDate=max(candidates.date).date()
  dateForecast = lastDate
  # Filter group for prod_id maintaining candidates dataset indexes
  group = candidates.loc[candidates['cluster_id'] == prod_id,:]
  # identify label changing
  for row in group.iterrows():
    stage = group.iloc[:,3][row[0]]
    DiffList.append(row[0])
    if test:
      print(row[0],' Last stage:', last_stage, '-- Stage:', stage, '...' , sep='')
    if (stage != last_stage) and (last_stage != None) and ((stage == 'Maturity') or (stage == 'Second_Maturity')):
      if test:
        print(row[0],' Last stage:', last_stage, '-- Stage:', stage, '...' , sep='')
      idxTransitions.append(row[0])
      print(row[0],'Transition for id:', prod_id) #, candidates.iloc[:,5][row[0]])
    if (stage != last_stage) and (last_stage != 'Forecasting') and (stage == 'Forecasting'):
      dateForecast = candidates.iloc[row[0],2].date()
    last_stage = stage
    count += 1
  # Testing the submission
  if (len(idxTransitions)>0):
    prodList.iloc[prodList.loc[prodList['cluster_id'] == prod_id].index,1] = lastDate - relativedelta(months=36)
    prodList.iloc[prodList.loc[prodList['cluster_id'] == prod_id].index,2] = 'NO'
  else:
    prodList.iloc[prodList.loc[prodList['cluster_id'] == prod_id].index,1] = dateForecast + relativedelta(months=5)
    prodList.iloc[prodList.loc[prodList['cluster_id'] == prod_id].index,2] = 'YES'
  # Testing the submission
  prodList.iloc[prodList.loc[prodList['cluster_id'] == prod_id].index,5] = [DiffList]
  prodList.iloc[prodList.loc[prodList['cluster_id'] == prod_id].index,6] = [idxTransitions]
  if test:
    print('Interval for', prod_id, ' ,', ' transitions:', idxTransitions, ',', count, 'rows availables')
    minDifList= min(DiffList)
    maxDifList= max(DiffList)
    print(len(DiffList), 'Ordered indexes from', minDifList, 'to', maxDifList)
    print('Group elements:', DiffList)
  DiffST_LEN = 27
  DiffLT_LEN = 49
  if (len(idxTransitions) > 0):
    if test:
      print('Generating Intervals for first transition on', idxTransitions[0])
    DiffPos = DiffList.index(idxTransitions[0])
    if test:
      print('Indice general en dataset candidatos, y grupo de ID:', DiffList[DiffPos])
      print('Indice de transicion en intervalo de grupo:', DiffPos)
      print('DiffST interval:[', DiffST_LEN-DiffST_LEN//2, DiffST_LEN//2, ']', end='')
      print('----> [', DiffPos-(DiffST_LEN-DiffST_LEN//2), DiffPos+DiffST_LEN//2, ']')
      print('DiffLT interval:[', DiffLT_LEN-DiffLT_LEN//2, DiffLT_LEN//2, ']', end='')
      print('----> [', DiffPos-(DiffLT_LEN-DiffLT_LEN//2), DiffPos+DiffLT_LEN//2, ']')
    DiffST= DiffList[DiffPos-(DiffST_LEN-DiffST_LEN//2) : DiffPos+DiffST_LEN//2]
    if test:
      print('DiffST:',len(DiffST))
      print('DiffST', DiffST)
    DiffLT= DiffList[DiffPos-(DiffLT_LEN-DiffLT_LEN//2):DiffPos+DiffLT_LEN//2]
    if test:
      print('DiffLT:', len(DiffLT))
      print(DiffLT)
    # Also store on new dataframe idxST_l and idxLT_l associated to ID_xxxx would be nice
    # prod_id<--->idxLtrans<--->[idxST_l]<--->[idxLT_l]
    prodList.iloc[prodList.loc[prodList['cluster_id'] == prod_id].index,8] = [DiffST]
    prodList.iloc[prodList.loc[prodList['cluster_id'] == prod_id].index,9] = [DiffLT]

In [27]:
prod_id = 'ID_65'
scanProdId(prod_id)

3509 Transition for id: ID_65


In [28]:
print('Procesando toda la lista de productos...')
for id in prodList.iterrows():
  #print(prodList.iloc[:,0][id[0]])
  scanProdId(prodList.iloc[:,0][id[0]])

Procesando toda la lista de productos...
645 Transition for id: ID_12
923 Transition for id: ID_17
1017 Transition for id: ID_18
1265 Transition for id: ID_21
1414 Transition for id: ID_22
1513 Transition for id: ID_23
1761 Transition for id: ID_25
1910 Transition for id: ID_27
1989 Transition for id: ID_31
2109 Transition for id: ID_32
2230 Transition for id: ID_33
2649 Transition for id: ID_43
2833 Transition for id: ID_45
2931 Transition for id: ID_60
3028 Transition for id: ID_61
3355 Transition for id: ID_64
3509 Transition for id: ID_65
4183 Transition for id: ID_93
4607 Transition for id: ID_102
4807 Transition for id: ID_106
4954 Transition for id: ID_107
5054 Transition for id: ID_108
5127 Transition for id: ID_109
5227 Transition for id: ID_110
5613 Transition for id: ID_115
6250 Transition for id: ID_126
6614 Transition for id: ID_144
6843 Transition for id: ID_163
6982 Transition for id: ID_166
7230 Transition for id: ID_168
7458 Transition for id: ID_173
7850 Transition fo

##### Calculate derivatives for each group

In [29]:
# Calculate the pseudo derivative into ST and LT index intervals

In [30]:
# insert values into EvalDiff_ST and EvalDiff_LT columns

# Testing features

##### Valueref, salesNormCoefficient, norm_mean

##### Differrential, or incremental difference`

In [31]:
candidates.volume.head()

0    1.145774e+07
1    2.985987e+09
2    1.984680e+09
3    6.267384e+09
4    6.293787e+09
Name: volume, dtype: float64

In [32]:
candidates.volume.diff().head(20)

0              NaN
1     2.974529e+09
2    -1.001307e+09
3     4.282704e+09
4     2.640262e+07
5    -1.801356e+09
6     2.297526e+09
7    -3.783545e+09
8     1.706705e+09
9    -1.488012e+09
10    3.105546e+09
11   -3.285881e+09
12    1.219004e+09
13   -2.309482e+09
14    2.246713e+09
15    4.971663e+08
16    1.092969e+09
17   -3.670960e+09
18    2.324925e+09
19    3.105048e+09
Name: volume, dtype: float64

##### Normalization of sales

In [33]:
# Determine best valueRef for Dataframe, and test feature
print(normalizeSales(candidates.volume,1))

Range for all registers to fit valueref: 24409918583946
real mean calculated 83165554307.12386
(4.096695351772633e-14, 0.003407039397575888)


In [34]:
# Once cheched, validate that with new valueRef coefficient is 1.0
print(normalizeSales(candidates.volume,24409918583946))

Range for all registers to fit valueref: 24409918583946
real mean calculated 83165554307.12386
(1.0, 83165554307.12386)


##### candidates dataFrame analysis

In [35]:
candidates.head()

,Unnamed: 0,cluster_id,date,stage_name,volume
0,248,ID_3,2012-01-01,Maturity,1.145774e+07
1,249,ID_3,2012-02-01,Maturity,2.985987e+09
2,250,ID_3,2012-03-01,Maturity,1.984680e+09
3,251,ID_3,2012-04-01,Maturity,6.267384e+09
4,252,ID_3,2012-05-01,Maturity,6.293787e+09


In [36]:
candidates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147397 entries, 0 to 147396
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  147397 non-null  int64         
 1   cluster_id  147397 non-null  object        
 2   date        147397 non-null  datetime64[ns]
 3   stage_name  147397 non-null  object        
 4   volume      147397 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 5.6+ MB


In [37]:
candidates['cluster_id'].describe()

count     147397
unique      1464
top         ID_3
freq         124
Name: cluster_id, dtype: object

In [38]:
candidates['date'].describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                  147397
unique                    124
top       2022-04-01 00:00:00
freq                     1464
first     2012-01-01 00:00:00
last      2022-04-01 00:00:00
Name: date, dtype: object

In [39]:
candidates['volume'].describe()

count    1.473970e+05
mean     8.316555e+10
std      5.787569e+11
min     -1.000000e+00
25%     -1.000000e+00
50%      2.368016e+07
75%      4.389145e+09
max      2.440992e+13
Name: volume, dtype: float64

In [40]:
candidates['stage_name'].describe()

count          147397
unique              8
top       Forecasting
freq            54168
Name: stage_name, dtype: object

In [41]:
candidates['stage_name'].unique()

array(['Maturity', 'Forecasting', 'Decline', 'Second_Growth',
       'Second_Maturity', 'Growth', 'Lead_in_Maturity', 'Lead_in_Decline'],
      dtype=object)

In [42]:
candidates.head()

,Unnamed: 0,cluster_id,date,stage_name,volume
0,248,ID_3,2012-01-01,Maturity,1.145774e+07
1,249,ID_3,2012-02-01,Maturity,2.985987e+09
2,250,ID_3,2012-03-01,Maturity,1.984680e+09
3,251,ID_3,2012-04-01,Maturity,6.267384e+09
4,252,ID_3,2012-05-01,Maturity,6.293787e+09


##### Differential Grouth to Mature, stage_name differences (string)
Actually discarded but interesting

In [43]:
# Create new column with stage change, but not good to fullfill almost with groups
#candidates_usefull_data['new_stage'] = bool
#candidates_usefull_data['new_stage'] = candidates_usefull_data['new_stage'].ne(candidates_usefull_data['new_stage'].shift().bfill()).astype(bool)

##### Try to select evaluation intervals to calculate pseudo-derivative.
Actually discarded

In [44]:
# Only add one time or clear values. Discarded for now, to implement on cluster_id list dataframe (prodList)
#candidates['EvalDiff_ST']= 0

In [45]:
# Only add one time or clear values. Discarded for now, to implement on cluster_id list dataframe (prodList)
#candidates['EvalDiff_LT']= 0

##### Create product list (prodList) from candidates

In [46]:
# Value test for index on group: would be same on candidates
group.iloc[:,0][147349] #-> Output 188688

188677

In [47]:
# Value test for index on candidates: would be same on group
candidates.iloc[:,0][147349] #-> Output 188688

188677

In [48]:
candidates.iloc[87,2].date()

datetime.date(2019, 4, 1)

In [49]:
prodList.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1464 entries, 0 to 1463
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cluster_id       1464 non-null   object
 1   transition_date  1464 non-null   object
 2   is_transition    1464 non-null   object
 3   date             1464 non-null   object
 4   forecast         1464 non-null   object
 5   idxGroup         1464 non-null   object
 6   idxLtrans        1464 non-null   object
 7   idxPtrans        1464 non-null   object
 8   idxST_l          1464 non-null   object
 9   idxLT_l          1464 non-null   object
 10  diffST_l         1464 non-null   object
 11  diffLT_l         1464 non-null   object
dtypes: object(12)
memory usage: 137.4+ KB


In [50]:
prodList.describe()

,cluster_id,transition_date,is_transition,date,forecast,idxGroup,idxLtrans,idxPtrans,idxST_l,idxLT_l,diffST_l,diffLT_l
count,1464,1464,1464,1464,1464,1464,1464,1464,1464,1464,1464,1464
unique,1464,2,2,1,1,1464,631,1,486,468,1,1
top,ID_3,2019-09-01,YES,<class 'datetime.date'>,<class 'int'>,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",[],<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>
freq,1,834,834,1464,1464,1,834,1464,834,834,1464,1464


In [51]:
prodList.loc[prodList['cluster_id']== 'ID_3221']

,cluster_id,transition_date,is_transition,date,forecast,idxGroup,idxLtrans,idxPtrans,idxST_l,idxLT_l,diffST_l,diffLT_l
1463,ID_3221,2019-04-01,NO,<class 'datetime.date'>,<class 'int'>,"[147310, 147311, 147312, 147313, 147314, 14731...",[147349],<class 'list'>,"[147335, 147336, 147337, 147338, 147339, 14734...","[147324, 147325, 147326, 147327, 147328, 14732...",<class 'list'>,<class 'list'>


In [52]:
prodList.tail()

,cluster_id,transition_date,is_transition,date,forecast,idxGroup,idxLtrans,idxPtrans,idxST_l,idxLT_l,diffST_l,diffLT_l
1459,ID_3212,2019-09-01,YES,<class 'datetime.date'>,<class 'int'>,"[147032, 147033, 147034, 147035, 147036, 14703...",[],<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>
1460,ID_3214,2019-09-01,YES,<class 'datetime.date'>,<class 'int'>,"[147096, 147097, 147098, 147099, 147100, 14710...",[],<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>
1461,ID_3215,2019-09-01,YES,<class 'datetime.date'>,<class 'int'>,"[147157, 147158, 147159, 147160, 147161, 14716...",[],<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>
1462,ID_3218,2019-09-01,YES,<class 'datetime.date'>,<class 'int'>,"[147220, 147221, 147222, 147223, 147224, 14722...",[],<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>
1463,ID_3221,2019-04-01,NO,<class 'datetime.date'>,<class 'int'>,"[147310, 147311, 147312, 147313, 147314, 14731...",[147349],<class 'list'>,"[147335, 147336, 147337, 147338, 147339, 14734...","[147324, 147325, 147326, 147327, 147328, 14732...",<class 'list'>,<class 'list'>


In [53]:
group

,Unnamed: 0,cluster_id,date,stage_name,volume
147310,188638,ID_3221,2015-02-01,Growth,1.992650e+06
147311,188639,ID_3221,2015-03-01,Growth,2.241732e+06
147312,188640,ID_3221,2015-04-01,Growth,1.743569e+06
147313,188641,ID_3221,2015-05-01,Growth,9.963252e+05
147314,188642,ID_3221,2015-06-01,Growth,9.714171e+06
147315,188643,ID_3221,2015-07-01,Growth,1.120866e+07
147316,188644,ID_3221,2015-08-01,Growth,1.444672e+07
147317,188645,ID_3221,2015-09-01,Growth,7.472439e+06
147318,188646,ID_3221,2015-10-01,Growth,1.843202e+07
147319,188647,ID_3221,2015-11-01,Growth,7.970602e+06


##### Create group to process each prod_id from cluster_id

##### Detect evaluation intervals by EvalDiff and list positions group by cluster_id

In [54]:
#prodList.loc[prodList['cluster_id'] == prod_id]

In [55]:
#prodList.iloc[prodList.loc[prodList['cluster_id'] == prod_id].index,5] = [DiffList]

In [56]:
#prodList.iloc[prodList.loc[prodList['cluster_id'] == prod_id].index,6].tolist

In [57]:
# Select element by index. Examples
prodList.loc[prodList['cluster_id']== 'ID_65'].idxGroup.iloc[0][0]

3454

In [58]:
# Select element by index. Examples
prodList.loc[prodList['cluster_id'].isin(['ID_65'])].idxLtrans.iloc[0][0]

3509

##### Create a submission1 csv file from prodList

In [59]:
!ls -l metric_one.csv

-rw-r--r-- 1 root root 73088 Nov 28 21:07 metric_one.csv


In [60]:
!head metric_one.csv

,cluster_id,transition_date,is_transition
0,ID_3,<class 'datetime.date'>,<class 'str'>
1,ID_5,<class 'datetime.date'>,<class 'str'>
2,ID_8,<class 'datetime.date'>,<class 'str'>
3,ID_9,<class 'datetime.date'>,<class 'str'>
4,ID_11,<class 'datetime.date'>,<class 'str'>
5,ID_12,<class 'datetime.date'>,<class 'str'>
6,ID_16,<class 'datetime.date'>,<class 'str'>
7,ID_17,<class 'datetime.date'>,<class 'str'>
8,ID_18,<class 'datetime.date'>,<class 'str'>


In [61]:
!tail metric_one.csv

1454,ID_3201,<class 'datetime.date'>,<class 'str'>
1455,ID_3202,<class 'datetime.date'>,<class 'str'>
1456,ID_3204,<class 'datetime.date'>,<class 'str'>
1457,ID_3207,<class 'datetime.date'>,<class 'str'>
1458,ID_3211,<class 'datetime.date'>,<class 'str'>
1459,ID_3212,<class 'datetime.date'>,<class 'str'>
1460,ID_3214,<class 'datetime.date'>,<class 'str'>
1461,ID_3215,<class 'datetime.date'>,<class 'str'>
1462,ID_3218,<class 'datetime.date'>,<class 'str'>
1463,ID_3221,<class 'datetime.date'>,<class 'str'>


#### Create a submission2 csv file from prodList

In [62]:
!ls -l metric_two.csv

-rw-r--r-- 1 root root 73072 Nov 28 21:07 metric_two.csv


In [63]:
!head metric_two.csv

,cluster_id,date,forecast
0,ID_3,<class 'datetime.date'>,<class 'int'>
1,ID_5,<class 'datetime.date'>,<class 'int'>
2,ID_8,<class 'datetime.date'>,<class 'int'>
3,ID_9,<class 'datetime.date'>,<class 'int'>
4,ID_11,<class 'datetime.date'>,<class 'int'>
5,ID_12,<class 'datetime.date'>,<class 'int'>
6,ID_16,<class 'datetime.date'>,<class 'int'>
7,ID_17,<class 'datetime.date'>,<class 'int'>
8,ID_18,<class 'datetime.date'>,<class 'int'>


In [64]:
!tail metric_two.csv

1454,ID_3201,<class 'datetime.date'>,<class 'int'>
1455,ID_3202,<class 'datetime.date'>,<class 'int'>
1456,ID_3204,<class 'datetime.date'>,<class 'int'>
1457,ID_3207,<class 'datetime.date'>,<class 'int'>
1458,ID_3211,<class 'datetime.date'>,<class 'int'>
1459,ID_3212,<class 'datetime.date'>,<class 'int'>
1460,ID_3214,<class 'datetime.date'>,<class 'int'>
1461,ID_3215,<class 'datetime.date'>,<class 'int'>
1462,ID_3218,<class 'datetime.date'>,<class 'int'>
1463,ID_3221,<class 'datetime.date'>,<class 'int'>
